<a href="https://colab.research.google.com/github/AtheerAllogamani/AtheerAllogamani/blob/main/ROCK_PAPER_SCISSORS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from IPython.display import Image
import numpy as np
from PIL import Image, ImageOps
from tensorflow.keras.models import load_model
import random
import warnings

image_path_scissors = "/content/Pictures/scissors.png"
image_path_rock = "/content/Pictures/rock.png"
image_path_paper = "/content/Pictures/paper.png"
options = ("rock", "paper", "scissors")

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = 300;
      canvas.height = 300;
      canvas.getContext('2d').drawImage(video, 0, 0, 300, 300);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

running = True

while running:

    player = None
    computer = random.choice(options)

    while player not in options:
        filename = take_photo()

        # Suppress warning messages
        warnings.simplefilter('ignore')

        # Load the trained model
        model = load_model('keras_model.h5', compile=True)

        # Define the labels for the classes
        labels = ['rock', 'paper', 'scissors']

        # Load the image and convert it to RGB
        image = Image.open(format(filename)).convert("RGB")

        # Resize the image to 224x224 and center crop it
        size = (224, 224)
        image = ImageOps.fit(image, size, Image.ANTIALIAS)

        # Convert the image to a NumPy array and normalize it
        image_array = np.asarray(image)
        normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

        # Load the image into the array
        data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
        data[0] = normalized_image_array

        # Run the inference
        prediction = model.predict(data)

        # Get the index of the class with the highest probability
        predicted_class_index = np.argmax(prediction)

        # Get the label for the predicted class
        predicted_label = labels[predicted_class_index]

        player = predicted_label


    if computer == "rock":
      image_path = image_path_rock
    elif computer == "paper":
      image_path = image_path_paper
    else:
      image_path = image_path_scissors

    print("Player: ")
    display(Image.open(format(filename)))
    print("Computer: ")
    display(Image.open(format(image_path)))


    if player == computer:
        print("It's a tie!")
    elif player == "rock" and computer == "scissors":
        print("You win!")
    elif player == "paper" and computer == "rock":
        print("You win!")
    elif player == "scissors" and computer == "paper":
        print("You win!")
    else:
        print("You lose!")

    if input("Play again? (y/n): ").lower() == "y" or "yes":
        running = False

print("Thanks for playing!")

<IPython.core.display.Javascript object>

OSError: ignored